KMRD 는 네이버 영화 평점을 바탕으로 MovieLens style 로 만든 데이터셋입니다. https://github.com/lovit/kmrd 에서 다운 받은 뒤, setup 으로 설치할 수 있습니다. Pandas DataFrame 을 이용하여 table join 을 연습하는 것이 목적이므로 데이터셋의 크기는 가장 작은 `small` 로 설정합니다. `get_paths()` 함수를 실행하면 데이터셋의 파일들의 주소가 list of str 로 return 됩니다.

In [1]:
import kmr_dataset
from kmr_dataset import get_paths
import numpy as np
import pandas as pd


print(f'kmr_dataset={kmr_dataset.__version__}\n')

paths = get_paths(size='small')
for i, path in enumerate(paths):
    print(f"{i}: {path.split('/')[-1]}")

kmr_dataset=1.0.1

0: castings.csv
1: countries.csv
2: genres.csv
3: movies.txt
4: peoples.txt
5: rates.csv


`*.txt` 는 tap separated, `*.csv` 는 comma separated 파일입니다. `pandas.read_csv()` 함수를 이용하여 데이터를 읽습니다.

In [2]:
movies = pd.read_csv(paths[3], delimiter='\t')
print(movies['movie'].dtype)
movies.head()

int64


,movie,title,title_eng,year,grade
0,10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가
1,10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가
2,10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가
3,10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가
4,10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG


영화의 연도가 float 형식입니다. 이를 int 로 변환활 것입니다. 그러나 영화의 연도가 기록되지 않은 경우들이 있습니다. 이는 NaN 으로 입력되었기 때문에 `astype()` 함수를 곧바로 실행할 수 없습니다. NaN 인 영화들이 언제 이용될지 모르니 이 값을 -1 로 변경한 뒤 연도의 형식을 int 로 변환합니다. 혹은 NaN 이 포함된 열을 제거하려면 `dropna()` 함수를 이용할 수도 있습니다.

In [3]:
# movies = movies.dropna(axis=0) 
movies = movies.fillna(-1)
movies = movies.astype({'year': 'int'})
movies.head()

,movie,title,title_eng,year,grade
0,10001,시네마 천국,"Cinema Paradiso , 1988",2013,전체 관람가
1,10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015,12세 관람가
2,10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015,12세 관람가
3,10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990,전체 관람가
4,10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997,PG


평점 파일은 (사용자, 영화, 점수, 시간) 형식입니다.

In [4]:
rates = pd.read_csv(paths[5], delimiter=',')
print(rates['movie'].dtype)
rates.head()

int64


,user,movie,rate,time
0,0,10003,7,1494128040
1,0,10004,7,1467529800
2,0,10018,9,1513344120
3,0,10021,9,1424497980
4,0,10022,7,1427627340


`groupby()` 함수를 이용하여 고유 (영화, 점수) 의 행의 개수를 계산합니다. 결과값의 형식은 Series 입니다.

In [5]:
rates.groupby(['movie', 'rate']).size()

movie  rate
10001  1       152
       2        14
       3        15
       4        10
       5        30
              ... 
10998  6         5
       7         4
       8        21
       9        20
       10       86
Length: 4507, dtype: int64

이 값을 `count` 라는 새로운 column 으로 만들고, `reset_index()` 함수를 실행하여 (영화, 점수, 개수) 의 테이블을 만듭니다.

In [6]:
count_row = pd.DataFrame(
    {'count': rates.groupby(['movie', 'rate']).size()}
)
count_row = count_row.reset_index()
count_row.head()

,movie,rate,count
0,10001,1,152
1,10001,2,14
2,10001,3,15
3,10001,4,10
4,10001,5,30


`pivot()` 함수를 이용하여 pivot table 을 만듭니다. (`movie`, `rate`) 의 값이 없는 경우는 NaN 으로 출력되므로 이 값을 0 으로 채웁니다.

In [7]:
# count = count_row.pivot(index='movie', columns='rate', values='count').reset_index()
count = count_row.pivot(index='movie', columns='rate', values='count').reset_index().fillna(0)
count.head()

rate,movie,1,2,3,4,5,6,7,8,9,10
0,10001,152.0,14.0,15.0,10.0,30.0,38.0,52.0,112.0,238.0,2974.0
1,10002,96.0,3.0,0.0,5.0,15.0,22.0,45.0,152.0,232.0,2434.0
2,10003,29.0,0.0,0.0,3.0,5.0,14.0,34.0,75.0,109.0,720.0
3,10004,11.0,1.0,4.0,5.0,5.0,21.0,44.0,69.0,72.0,468.0
4,10005,30.0,2.0,2.0,5.0,7.0,24.0,41.0,83.0,111.0,600.0


영화 별 점수 개수도 Series 로 만들어 둡니다.

In [8]:
count_sum_series = count.drop(['movie'], axis=1).sum(axis=1) # type Series
count_sum_series.head()

0    3635.0
1    3004.0
2     989.0
3     700.0
4     905.0
dtype: float64

(영화, 점수 개수) 의 DataFrame 도 만듭니다.

In [9]:
count_sum = pd.DataFrame(count_sum_series, columns=['count'])
count_sum['movie'] = count['movie']
# swap columns
count_sum = count_sum[['movie', 'count']]
count_sum.head()

,movie,count
0,10001,3635.0
1,10002,3004.0
2,10003,989.0
3,10004,700.0
4,10005,905.0


`count` 의 1 부터 10까지의 값에 대하여 각 행의 총합으로 나눠 이를 비율로 만듭니다.

In [10]:
proportion = count.loc[:,range(1,11)].div(count_sum_series, axis=0)
proportion['movie'] = count['movie']
proportion.head()

rate,1,2,3,4,5,6,7,8,9,10,movie
0,0.041816,0.003851,0.004127,0.002751,0.008253,0.010454,0.014305,0.030812,0.065475,0.818157,10001
1,0.031957,0.000999,0.000000,0.001664,0.004993,0.007324,0.014980,0.050599,0.077230,0.810253,10002
2,0.029323,0.000000,0.000000,0.003033,0.005056,0.014156,0.034378,0.075834,0.110212,0.728008,10003
3,0.015714,0.001429,0.005714,0.007143,0.007143,0.030000,0.062857,0.098571,0.102857,0.668571,10004
4,0.033149,0.002210,0.002210,0.005525,0.007735,0.026519,0.045304,0.091713,0.122652,0.662983,10005


영화 별 점수 분포를 만들기 위하여 `pandas.merge()` 함수를 이용합니다.

In [11]:
movies_proportion = pd.merge(
    movies.drop(['title_eng'], axis=1),
    proportion,
    how = 'right'
)
movies_proportion.head()

,movie,title,year,grade,1,2,3,4,5,6,7,8,9,10
0,10001,시네마 천국,2013,전체 관람가,0.041816,0.003851,0.004127,0.002751,0.008253,0.010454,0.014305,0.030812,0.065475,0.818157
1,10002,빽 투 더 퓨쳐,2015,12세 관람가,0.031957,0.000999,0.000000,0.001664,0.004993,0.007324,0.014980,0.050599,0.077230,0.810253
2,10003,빽 투 더 퓨쳐 2,2015,12세 관람가,0.029323,0.000000,0.000000,0.003033,0.005056,0.014156,0.034378,0.075834,0.110212,0.728008
3,10004,빽 투 더 퓨쳐 3,1990,전체 관람가,0.015714,0.001429,0.005714,0.007143,0.007143,0.030000,0.062857,0.098571,0.102857,0.668571
4,10005,스타워즈 에피소드 4 - 새로운 희망,1997,PG,0.033149,0.002210,0.002210,0.005525,0.007735,0.026519,0.045304,0.091713,0.122652,0.662983


## 연도 별 개봉 장르 개수

연도 별 개봉한 영화의 장르 개수를 테이블로 만들어봅니다. 최종 결과는 다음의 항목으로 이뤄진 테이블입니다.

1. 연도
2. 장르
3. 연도 별 장르 개수

In [12]:
genres = pd.read_csv(paths[2], delimiter=',')
print(genres['movie'].dtype)
genres.head()

int64


,movie,genre
0,10001,드라마
1,10001,멜로/로맨스
2,10002,SF
3,10002,코미디
4,10003,SF


In [13]:
genre_years = pd.merge(genres, movies[['movie', 'year']], how='left')
genre_years.head()

,movie,genre,year
0,10001,드라마,2013
1,10001,멜로/로맨스,2013
2,10002,SF,2015
3,10002,코미디,2015
4,10003,SF,2015


In [14]:
# Do something

Results

![](./figures/pandas_lecture_result0.png)

## 주연 배우, 연도 별 누적 평점 개수

영화에 주연으로 출연한 배우들의 연도 별 영화의 누적 평점 개수를 테이블로 만들어 봅니다. 최종 결과는 다음의 항목으로 이뤄진 테이블입니다.

1. 배우 아이디
1. 배우 이름
1. 연도
1. 해당 연도의 주연으로 출연한 영화에 달린 평점 개수

In [15]:
# (영화 아이디, 배우 아이디, 크레딧 순서, 주연배우 {1: 주연, 0: 조연})
castings = pd.read_csv(paths[0], delimiter=',')
castings.head()

,movie,people,order,leading
0,10001,4374,1,1
1,10001,178,2,1
2,10001,3241,3,1
3,10001,47952,4,1
4,10001,47953,5,0


In [16]:
peoples = pd.read_csv(paths[4], delimiter='\t')
peoples.head()

,people,korean,original
0,5,아담 볼드윈,Adam Baldwin
1,8,애드리안 라인,Adrian Lyne
2,9,에이단 퀸,Aidan Quinn
3,13,구로사와 아키라,Akira Kurosawa
4,15,알 파치노,Al Pacino


In [17]:
# Do something

Results

![](./figures/pandas_lecture_result1.png)